In [6]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

### Load Vocab

In [7]:
vocab = {}
idToWord = {}
with open('vocab.txt') as f:
    words = f.read().splitlines()
    for wordIndex in words:
        word, index = wordIndex.split(' -----> ')
        vocab[word] = index
        idToWord[int(index)] = word

### Create input squences

In [8]:
look_back_len = 1000 + 1
sequences = []
vocabulary_size = len(vocab)
print(vocabulary_size)

with open('int-seq.txt') as f:
    files = f.read().splitlines()
    for file in files:
        numbers = list(map(int, file.split(',')[:-1]))
        #print(numbers)
        for i in range(look_back_len, len(numbers)):
            seq = numbers[i-look_back_len:i]
            sequences.append(seq)
    #print(sequences)

    n_sequences = np.empty([len(sequences), look_back_len], dtype='int32')
    for i in range(len(sequences)):
        n_sequences[i] = sequences[i]

5002


### Convert output to one hot encoded vector

In [9]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
print(len(train_targets))

train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
#print(train_targets[0])

38318


### Train Model

In [10]:
import keras
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save("model_{}.hd5".format(epoch))

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

saver = CustomSaver()

model.fit(train_inputs,train_targets,epochs=8,verbose=1, validation_split=0.3, callbacks=[saver])
model.save("mymodel.h5")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1000, 1000)        5002000   
_________________________________________________________________
lstm_7 (LSTM)                (None, 1000, 50)          210200    
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 5002)              255102    
Total params: 5,490,052
Trainable params: 5,490,052
Non-trainable params: 0
_________________________________________________________________
None


c:\Miniconda\envs\azureml_py36_tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 26822 samples, validate on 11496 samples
Epoch 1/8
26822/26822 [==============================] - 4961s 185ms/step - loss: 4.1742 - accuracy: 0.1936 - val_loss: 3.3540 - val_accuracy: 0.3559
Epoch 2/8
26822/26822 [==============================] - 5069s 189ms/step - loss: 3.5030 - accuracy: 0.3127 - val_loss: 3.1373 - val_accuracy: 0.4410
Epoch 3/8
26822/26822 [==============================] - 5134s 191ms/step - loss: 3.1565 - accuracy: 0.3905 - val_loss: 3.0404 - val_accuracy: 0.4733
Epoch 4/8
26822/26822 [==============================] - 5192s 194ms/step - loss: 2.8957 - accuracy: 0.4328 - val_loss: 3.0421 - val_accuracy: 0.5003
Epoch 5/8
26822/26822 [==============================] - 5224s 195ms/step - loss: 2.6910 - accuracy: 0.4702 - val_loss: 2.9957 - val_accuracy: 0.5045
Epoch 6/8
18560/26822 [===================>..........] - ETA: 24:45 - loss: 2.5369 - accuracy: 0.4942

### Test Model

In [24]:
from keras.preprocessing.sequence import pad_sequences

# sample python input
"""
num = 46
print
"""
encoded_text = [2,13,35,5]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre', value = int(vocab['<pad_token>']))
print(encoded_text, pad_encoded)

print("Top 3 Suggestions:")
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    print(idToWord[i])

[2, 13, 35, 5] [[36  2 13 35  5]]
Top 3 Suggestions:
num
1
n
